In [1]:
import sys
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import os
import itertools
import socket
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import time
import holidays
import pycountry

import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from IPython.display import display
from IPython.display import clear_output

In [2]:
# Declare important variables
train_path = 'train.csv'
solve_path = 'test.csv'
target_variable = 'num_sold'
SEED = 99
skip_hypertuning = True
cat_encoder_type = 'OneHotEncoder' #OneHotEncoder, FrequencyEncoder, None

In [3]:
def load_dataframes(path1: str, path2: str, format1: str = "csv", format2: str = "csv") -> tuple:

    loaders = {
        "csv": pd.read_csv,
        "excel": pd.read_excel,
        "json": pd.read_json,
    }

    if format1 not in loaders or format2 not in loaders:
        raise ValueError("Unsupported format. Supported formats: 'csv', 'excel', 'json'.")

    # Load the dataframes using appropriate loaders
    df1 = loaders[format1](path1)
    df2 = loaders[format2](path2)

    print(f"Loading data from {os.getcwd()}")
    
    return df1, df2

In [4]:
df_train_original, df_solve_original = load_dataframes(train_path,solve_path)

Loading data from C:\Users\ZaneC\OneDrive\Documents\Python\Kaggle\Playground Series\s5e1 - Sticker Sales


In [5]:
df_train_id = df_train_original['id']
df_train = df_train_original.drop('id',axis=1)

df_solve_id = df_solve_original['id']
df_solve = df_solve_original.drop('id',axis=1)

In [6]:
from typing import Tuple

def plot_kde_grid(df: pd.DataFrame, n_cols: int = 4, figsize: Tuple[int, int] = (12,8)) -> None:
    """
    Plots KDE for all numeric columns in the dataframe in a grid layout.
    
    Parameters:
        df (pd.DataFrame): DataFrame with numeric columns to plot.
        n_cols (int): Number of columns in the grid (default is 3).
        figsize (tuple): Size of the entire figure (default is (12, 8)).
    
    Returns:
        None
    """
    # Get numeric columns
    numeric_cols = df.select_dtypes(include='number').columns
    
    # Set up grid size
    n_rows = int(np.ceil(len(numeric_cols) / n_cols))  # Number of rows based on the number of features
    
    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(figsize))  # Adjust figure size
    axes = axes.flatten()  # Flatten to easily iterate
    
    # Plot each numeric column
    for i, col in enumerate(numeric_cols):
        sns.kdeplot(data=df, x=col, fill=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Density')
    
    # Hide any unused subplots
    for i in range(len(numeric_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [7]:
#plot_kde_grid(df_train,n_cols=4,figsize=(12,8))

In [8]:
#plot_kde_grid(df_solve,n_cols=4,figsize=(9,6))

In [9]:
def describe_dataframe(df_to_use: pd.DataFrame, name: str = 'DataFrame', nrows: int = 3, plots: bool = False, info: bool = True) -> None:
    '''
    Function to describe the DataFrame with summary statistics, missing value count,
    unique value count, and duplicate count. It also displays plots for missing and unique values.
    
    Parameters:
    - df: The DataFrame to describe.
    - name: The name to display in the summary (default is 'DataFrame').
    - nrows: The number of rows to display from the top and bottom (default is 3).
    - plots: Whether to display bar plots for missing and unique values (default is False).
    - info: Whether to display the styled DataFrame (default is True).
    '''

    df = df_to_use.copy()
    
    # Create summary DataFrame
    inf = pd.DataFrame(df.dtypes).reset_index().rename(columns={'index':'column', 0:'type'})
    
    # Missing values
    df_missing = pd.DataFrame(df.isnull().sum()).reset_index().rename(columns={'index':'column', 0:'missing'})
    df_missing['pct_missing'] = (df_missing['missing'] / df.shape[0]) * 100
    
    # Unique values
    df_unique = pd.DataFrame(df.nunique()).reset_index().rename(columns={'index':'column', 0:'unique'})
    
    # Combine summary information
    inf['missing'] = df_missing['missing']
    inf['pct_missing'] = df_missing['pct_missing']
    inf['unique'] = df_unique['unique']
    inf['duplicate'] = df.duplicated().sum()
    inf['count'] = df.shape[0]

    # Descriptive statistics
    desc = pd.DataFrame(df.describe(include='all').transpose())
    if 'min' in desc.columns.tolist():
        inf['min'] = desc['min'].values
        inf['max'] = desc['max'].values
        inf['avg'] = desc['mean'].values
        inf['std dev'] = desc['std'].values
    if 'top' in desc.columns.tolist():
        inf['top value'] = desc['top'].values
        inf['Freq'] = desc['freq'].values 
    
    # Display styled DataFrame
    if info:
        display(inf.style.background_gradient(subset=['missing','pct_missing'], cmap='Reds').background_gradient(subset='unique', cmap='Greens'))

    if nrows != 0 :
        # Display top and bottom nrows of the DataFrame
        print(f"\n---------- {name} Overview ----------:")
        print(f"{name} has {df.shape[0]} rows and {df.shape[1]} columns\n")
        display(df.head(nrows))
        display(df.tail(nrows))
    
    # Plot missing values if any
    if plots and df_missing['missing'].sum() > 0:
        fig, ax = plt.subplots(1, 1, figsize=(24, 5))
        sns.barplot(df_missing[df_missing['missing'] > 0], x='column', y='missing', ax=ax)
        ax.set_title(f'{name} missing Values') 
        ax.bar_label(ax.containers[0])
        plt.tight_layout()
        plt.show()
    
        # Plot unique values
        fig, ax = plt.subplots(1, 1, figsize=(24, 5))
        sns.barplot(df_unique[df_unique['unique'] > 0], x='column', y='unique', ax=ax)
        ax.set_title(f'{name} Unique Values')
        ax.bar_label(ax.containers[0])
        plt.tight_layout()
        plt.show()

In [10]:
def get_num_cols(df: pd.DataFrame):
    num_cols = df.select_dtypes(include=['number']).columns.to_list()
    
    return num_cols

In [11]:
def get_cat_cols(df: pd.DataFrame):
    cat_cols = df.select_dtypes(include=['object', 'string', 'category']).columns.tolist()  

    return cat_cols

In [12]:
def clean_dataframe(df_to_use: pd.DataFrame, impute_num_nulls: bool = True, fill_cat_nulls: bool = True) -> None:
    df = df_to_use.copy()

    # Convert date column from object to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter columns where the type is either 'float' or 'int' and there are missing values
    numeric_cols = df.select_dtypes(include=['number']).columns
    missing_numeric_columns = [
        col for col in numeric_cols if df[col].isnull().sum() > 0
    ]

    # Target field 'num_sold' is missing some values, we do not want to impute them
    if 'num_sold' in missing_numeric_columns:
        missing_numeric_columns.remove('num_sold')
    
    # Fill nulls in numeric columns with the median
    if impute_num_nulls:       
        for column in missing_numeric_columns:
            mdn = df[column].median()
            df[column] = df[column].fillna(mdn)

    
    # Get category columns
    if fill_cat_nulls:
        cat_cols = df.select_dtypes(include=['object', 'string','category']).columns.tolist()  
        # Fill missing values in object columns
        for column in cat_cols:
            df[column] = df[column].fillna('None')
    
    return df

In [13]:
def transform_dataframe(df_to_use: pd.DataFrame):
    
    df = df_to_use.copy()


    def get_holiday_name(country_code, date_obj):
        country_holiday = holidays.CountryHoliday(country_code, years=date_obj.year)
        return country_holiday.get(date_obj)

    def get_country_code(country_name):
        try:
            country = pycountry.countries.get(name=country_name)
            return country.alpha_2  
        except KeyError:
            print(f"Unknown Country: {country_name}")
            return None
    
    def get_holiday_for_row(row):
        country_code = get_country_code(row['country'])
        if country_code is None:
            return 'Unknown Country'     
        try:
            date_obj = row['date']
        except ValueError:
            print(f"Invalid Date: {row['date']}")
            return 'Invalid Date'
        
        return get_holiday_name(country_code, date_obj)
      
    df['holiday'] = df.apply(get_holiday_for_row, axis=1)

    df['Year'] = df['date'].dt.year
    df['Quarter'] = df['date'].dt.quarter
    df['Month'] = df['date'].dt.month
    df['Day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.day_name()
    df['day_of_year'] = df['date'].dt.dayofyear
    df['week_of_year'] = df['date'].dt.isocalendar().week

    df['day_sin'] = np.sin(2 * np.pi * df['Day'] / 365.0)
    df['day_cos'] = np.cos(2 * np.pi * df['Day'] / 365.0)
    df['month_sin'] = np.sin(2 * np.pi * df['Month'] / 12.0)
    df['month_cos'] = np.cos(2 * np.pi * df['Month'] / 12.0)
    df['year_sin'] = np.sin(2 * np.pi * df['Year'] / 7.0)
    df['year_cos'] = np.cos(2 * np.pi * df['Year'] / 7.0)
    df['Group']=(df['Year']-2010)*48+df['Month']*4+df['Day']//7
    
    
    df['Quarter'] = df['Quarter'].astype('str')
    df['Month'] = df['Month'].astype('str')
    df['day_of_week'] = df['day_of_week'].astype('str')
    df['day_of_year'] = df['day_of_week'].astype('str')
    df['week_of_year'] = df['week_of_year'].astype('str')

    df = df.drop('date', axis=1)

    if 'num_sold' in df.columns:
        df = df.dropna(subset=['num_sold'])
    
    return df

In [14]:
# Clean and Transform Data

In [15]:
df_train_cleaned = clean_dataframe(df_train, impute_num_nulls = True, fill_cat_nulls=False)
df_solve_cleaned = clean_dataframe(df_solve, impute_num_nulls = True, fill_cat_nulls=False)

In [16]:
df_train_prepared = transform_dataframe(df_train_cleaned)
df_solve_prepared = transform_dataframe(df_solve_cleaned)

In [17]:
# Inspect Data through phases of clean and transform

In [18]:
describe_dataframe(df_train, name='Insurance Data', nrows=0, plots=False, info=True)

,column,type,missing,pct_missing,unique,duplicate,count,min,max,avg,std dev,top value,Freq
0,date,object,0,0.000000,2557,0,230130,nan,nan,nan,nan,2010-01-01,90
1,country,object,0,0.000000,6,0,230130,nan,nan,nan,nan,Canada,38355
2,store,object,0,0.000000,3,0,230130,nan,nan,nan,nan,Discount Stickers,76710
3,product,object,0,0.000000,5,0,230130,nan,nan,nan,nan,Holographic Goose,46026
4,num_sold,float64,8871,3.854778,4037,0,230130,5.000000,5939.000000,752.527382,690.165445,nan,nan


In [19]:
describe_dataframe(df_train_cleaned, name='Insurance Data', nrows=0, plots=False, info=True)

,column,type,missing,pct_missing,unique,duplicate,count,min,max,avg,std dev,top value,Freq
0,date,datetime64[ns],0,0.000000,2557,0,230130,2010-01-01 00:00:00,2016-12-31 00:00:00,2013-07-02 00:00:00,nan,nan,nan
1,country,object,0,0.000000,6,0,230130,nan,nan,nan,nan,Canada,38355
2,store,object,0,0.000000,3,0,230130,nan,nan,nan,nan,Discount Stickers,76710
3,product,object,0,0.000000,5,0,230130,nan,nan,nan,nan,Holographic Goose,46026
4,num_sold,float64,8871,3.854778,4037,0,230130,5.000000,5939.000000,752.527382,690.165445,nan,nan


In [20]:
describe_dataframe(df_train_prepared, name='Insurance Data', nrows=0, plots=False, info=True)

,column,type,missing,pct_missing,unique,duplicate,count,min,max,avg,std dev,top value,Freq
0,country,object,0,0.000000,6,0,221259,nan,nan,nan,nan,Finland,38355
1,store,object,0,0.000000,3,0,221259,nan,nan,nan,nan,Premium Sticker Mart,75684
2,product,object,0,0.000000,5,0,221259,nan,nan,nan,nan,Kaggle,46026
3,num_sold,float64,0,0.000000,4037,0,221259,5.000000,5939.000000,752.527382,690.165445,nan,nan
4,holiday,object,214311,96.859789,71,0,221259,nan,nan,nan,nan,New Year's Day,301
5,Year,int32,0,0.000000,7,0,221259,2010.000000,2016.000000,2013.010748,1.996617,nan,nan
6,Quarter,object,0,0.000000,4,0,221259,nan,nan,nan,nan,4,56194
7,Month,object,0,0.000000,12,0,221259,nan,nan,nan,nan,12,18994
8,Day,int32,0,0.000000,31,0,221259,1.000000,31.000000,15.728576,8.800895,nan,nan
9,day_of_week,object,0,0.000000,7,0,221259,nan,nan,nan,nan,Sunday,31958


In [21]:

 num_cols = get_num_cols(df_train_prepared)

# plt.figure(figsize=(20,10))
# sns.heatmap(df_train_prepared[num_cols].corr(),
#             vmin=-1,
#             vmax=1,
#             center=0,
#             annot=True,
#             cmap='PRGn')
# plt.show()

In [22]:
corr_col = df_train_prepared[num_cols].corr()[target_variable].drop([target_variable])
corr_df = corr_col.reset_index()
corr_df.columns = ['Columns', 'Correlation']
corr_df = corr_df.sort_values(ascending = False, by = 'Correlation')
corr_df

,Columns,Correlation
6,year_sin,0.018388
4,month_sin,0.014119
1,Day,0.001137
2,day_sin,0.000936
5,month_cos,-0.001781
3,day_cos,-0.003392
0,Year,-0.040462
8,Group,-0.040925
7,year_cos,-0.067782


In [23]:
from scipy.stats import f_oneway

def anova_test(data, cat_cols, target):
    anova_res = []
    for col in cat_cols:
        groups = [data[target][data[col] == cat] for cat in data[col].unique()]
        f_stat, p_val = f_oneway(*groups)
        anova_res.append([col, f_stat, p_val])
    return anova_res

In [24]:
cat_cols = get_cat_cols(df_train_prepared)
anova_res = anova_test(df_train_prepared, cat_cols, target_variable)
anova_df = pd.DataFrame(anova_res, columns = ['Column', 'F Statistic', 'P-value'])
anova_df = anova_df.sort_values(ascending = False, by = 'P-value')
anova_df

C:\Users\ZaneC\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4133: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '


,Column,F Statistic,P-value
4,Quarter,13.270418,1.170317e-08
5,Month,9.432666,3.240146e-17
7,week_of_year,5.530161,2.614399e-34
6,day_of_week,243.617487,1.150257e-311
0,country,24669.224785,0.000000e+00
1,store,13971.768812,0.000000e+00
2,product,19576.262467,0.000000e+00
3,holiday,NaN,NaN


In [25]:
#selected columns
# sel_cat = np.array(anova_df[anova_df['P-value'] < 0.05]['Column'])
# sel_num = np.array(corr_df[(corr_df['Correlation'] > 0.01) | (corr_df['Correlation'] < -0.01)]['Columns'])
# best_cols = np.append(sel_num, sel_cat)
best_cols = np.append(num_cols, cat_cols)

In [26]:
sns.set(rc={'figure.figsize':(24,8)})
ax=sns.lineplot(data=df_train_prepared,x='day_of_year',y='num_sold')
ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

ValueError: Could not interpret value `Day of Year` for parameter `x`

In [ ]:
sns.set(rc={'figure.figsize':(24,8)})
ax=sns.lineplot(data=df_train_prepared,x='week_of_year',y='num_sold')
ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
sns.set(rc={'figure.figsize':(24,8)})
ax=sns.lineplot(data=df_train_prepared,x='Month',y='num_sold')
ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
sns.set(rc={'figure.figsize':(24,8)})
ax=sns.lineplot(data=df_train_prepared,x='Year',y='num_sold', hue='country')
ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
sns.set(rc={'figure.figsize':(24,8)})
ax=sns.lineplot(data=df_train_prepared,x='Day of Year Sin',y='num_sold')
ax.axes.set_title("\nBasic Time Series of Sales\n",fontsize=20);

In [ ]:
# counts = X_test['holidays_name'].value_counts()
# counts.to_csv('test.csv')

In [27]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data

X = df_train_prepared.drop(f'{target_variable}', axis=1)
y = df_train_prepared[f'{target_variable}'].copy()

# Define test size and validation size relative to the entire dataset
test_size = 0.2
val_size = 0.2 / (1 - test_size)  # Validation size relative to training data

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=SEED)

# Create train/val split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, shuffle=True, random_state=SEED)


In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# import pandas as pd
# import numpy as np

# X = df_train_prepared.drop(f'{target_variable}', axis=1)
# y = df_train_prepared[f'{target_variable}'].copy()

# # Initialize TimeSeriesSplit with 5 splits
# tscv = TimeSeriesSplit(n_splits=5)

# # TimeSeriesSplit splits the data into train and test sets
# for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    

In [28]:
import numpy as np
from fitter import Fitter


# Initialize the Fitter object
f = Fitter(X_train)

# Fit data to distributions
f.fit()

# Display the summary of results
print(f.summary())

# Visualize the best fit
f.plot_pdf(names=f.get_best(method='sumsquare_error'))

TypeError: '<=' not supported between instances of 'str' and 'NoneType'

In [ ]:
print(X['holiday'].nunique())
print(X_train['holiday'].nunique())
print(X_val['holiday'].nunique())
print(X_test['holiday'].nunique())

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import scipy
import pandas as pd

def pipeline_fit(train_df: pd.DataFrame, cat_encoder: str = 'None'):
    
    # Define a custom transformer for frequency encoding
    class FrequencyEncoder(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns

        def fit(self, X, y=None):
            # Calculate the frequency for each category in the columns
            self.freq_dict = {}
            for col in self.columns:
                # Percentage-Based frequency
                self.freq_dict[col] = X[col].value_counts() / len(X)
            return self

        def transform(self, X):
            X_copy = X.copy()
            # Apply frequency encoding based on training set frequencies
            for col in self.columns:
                X_copy[col] = X_copy[col].map(self.freq_dict[col]).fillna(0)
            return X_copy

    # Identify numeric and categorical columns
    num_cols = get_num_cols(train_df)
    cat_cols = get_cat_cols(train_df)

    # Define a ColumnTransformer based on the specified encoding type
    if cat_encoder == 'OneHotEncoder':
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), num_cols),
                ('cat', OneHotEncoder(), cat_cols)
            ],
            remainder='passthrough'
        )
    elif cat_encoder == 'FrequencyEncoder':
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), num_cols),
                ('cat', FrequencyEncoder(columns=cat_cols), cat_cols)
            ],
            remainder='passthrough'
        )
    elif cat_encoder == 'None':
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), num_cols),
            ],
            remainder='passthrough'
        )
    else:
        raise ValueError("cat_encoder must be 'OneHotEncoder', 'FrequencyEncoder', or 'None'.")

    # Create a pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    # Fit and transform the train data
    X_train_prepared = pipeline.fit_transform(train_df)

    # Extract column names for the transformed DataFrame
    if cat_encoder == 'OneHotEncoder':
        num_feature_names = num_cols  # Numeric features retain their original names
        cat_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_cols)
        transformed_columns = list(num_feature_names) + list(cat_feature_names)
    elif cat_encoder == 'FrequencyEncoder' or cat_encoder == 'None':
        transformed_columns = num_cols + cat_cols

    # Convert sparse matrix to dense array
    if isinstance(X_train_prepared, scipy.sparse.spmatrix):
        X_train_prepared = X_train_prepared.toarray()

    # Convert the output to a DataFrame (All columns are OBJECT)
    X_train_prepared_df = pd.DataFrame(X_train_prepared, columns=transformed_columns)
    
    return X_train_prepared, X_train_prepared_df, pipeline


def pipeline_transform(test_df: pd.DataFrame, pipeline: Pipeline, cat_encoder: str = 'None'):
    # Transform the test data using the provided pipeline
    X_test_transformed = pipeline.transform(test_df)

    # Extract column names for the transformed DataFrame
    num_cols = get_num_cols(test_df)
    cat_cols = get_cat_cols(test_df)

    if cat_encoder == 'OneHotEncoder':
        num_feature_names = num_cols  # Numeric features retain their original names
        cat_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_cols)
        transformed_columns = list(num_feature_names) + list(cat_feature_names)
    elif cat_encoder == 'FrequencyEncoder' or cat_encoder == 'None':
        transformed_columns = num_cols + cat_cols
    
    # Convert sparse matrix to dense array
    if isinstance(X_test_transformed, scipy.sparse.spmatrix):
        X_test_transformed = X_test_transformed.toarray()
    
    # Convert the output to a DataFrame (All columns are OBJECT)
    X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=transformed_columns)

    return X_test_transformed, X_test_transformed_df




In [ ]:
# Fit the Pipeline
X_train_prepared, X_train_prepared_df, pipeline = pipeline_fit(X_train, cat_encoder = cat_encoder_type)
# Transform using the Pipeline
X_val_prepared, X_val_prepared_df = pipeline_transform(X_val, pipeline, cat_encoder = cat_encoder_type)
X_test_prepared, X_test_prepared_df = pipeline_transform(X_test, pipeline, cat_encoder = cat_encoder_type)

In [ ]:
from catboost import CatBoostClassifier
from catboost.utils import get_gpu_device_count

# Check available GPUs
gpu_count = get_gpu_device_count()
if gpu_count > 0:
    print(f"GPU is available with {gpu_count} GPU(s).")
else:
    print("No GPU detected. Ensure your setup is correct.")

# Test CatBoost with GPU
try:
    model = CatBoostClassifier(task_type="GPU", devices='0')  # Specify GPU
    device = 'gpu'
    print("CatBoost can use the GPU.")
        
except Exception as e:
    device = 'cpu'
    print(f"Error: {e}")

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from lightgbm import LGBMRegressor

# # Parameter grid
# param_grid = {
#     'num_leaves': [31, 60],       # Number of leaves in a tree
#     #'max_depth': [-1, 5, 10],        # limit on depth
#     'learning_rate': [0.01,.001], # Step size for updates
#     'n_estimators': [500, 750, 1000], # Number of boosting rounds
#     #'subsample': [0.8, 1.0],
#     #'colsample_bytree': [0.8, 1.0]
# }

# # Leave some cores available
# available_cores = os.cpu_count()
# n_cores = max(1, available_cores - 3)  # Leave 3 cores free


# lgbm = lgb.LGBMRegressor(random_state=SEED)

# # Set up GridSearchCV
# grid_search = GridSearchCV(
#     estimator=lgbm,
#     param_grid=param_grid,
#     scoring="neg_mean_squared_error",  # Use RMSE as the evaluation metric
#     cv=3,  # #-fold cross-validation
#     verbose=2,  # Output progress messages
#     n_jobs=n_cores  # Use all available cores
# )

# if skip_hypertuning:

#     best_params = {'learning_rate': 0.01, 'n_estimators': 500, 'num_leaves': 31}

# else:
#     start_time = time.time()                      
                          
#     grid_search.fit(X_train_prepared, y_log_train) #,feature_name=X.columns.tolist())
    
#     end_time = time.time()                      
                          
#     best_params = grid_search.best_params_
#     best_rmse = (-grid_search.best_score_) ** 0.5
    
#     print("Best parameters:",best_params)
#     print("Best RMSE:", best_rmse)
    
#     # Calculate elapsed time in minutes
#     elapsed_time_minutes = (end_time - start_time) / 60
#     print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")
    

In [ ]:
#####
# Try Bayesian Optimization
#### 
from skopt import BayesSearchCV
import lightgbm as lgb

# Hyperparameters for BayesSearchCV tuning
# Define the search space
search_space = {
    'n_estimators': (50, 500),                 # Number of boosting rounds
    'learning_rate': (0.01, 0.3, 'log-uniform'),  # Learning rate
    'max_depth': (3, 15),                     # Maximum depth of trees
    'num_leaves': (20, 150),                  # Number of leaves per tree
}

# Leave some cores available
available_cores = os.cpu_count()
n_cores = max(1, available_cores - 4)  # Leave 4 cores free


lgbm = lgb.LGBMRegressor(random_state=SEED)

# Set up BayesSearchCV for hyperparameter tuning
bayes_search  = BayesSearchCV(
    estimator=lgbm,
    search_spaces=search_space,
    n_iter=32,          # Number of iterations for optimization
    cv=3,               # k-fold cross-validation
    verbose=1,          # Display detailed logs
    scoring="neg_root_mean_squared_error",
    n_jobs = n_cores,
    random_state=SEED   
)

if skip_hypertuning:

    best_params = {'learning_rate': 0.02276243256597058,'max_depth' : 10, 'n_estimators' : 500,
                   'num_leaves' : 23}

else:
    start_time = time.time()
    
    bayes_search.fit(X_train_prepared, y_log_train)
    
    end_time = time.time()  
    
    best_params = bayes_search.best_params_
    best_rmse = (-bayes_search.best_score_) ** 0.5
    
    print("Best parameters:",best_params)
    print("Best RMSE:", best_rmse)

    # Calculate elapsed time in minutes
    elapsed_time_minutes = (end_time - start_time) / 60
    print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")

In [ ]:

# Initialize the LGBMRegressor
model = lgb.LGBMRegressor(
    **best_params,
    device=device,
    boosting_type='gbdt',  # Gradient Boosted Decision Trees
    random_state=SEED        # Ensure reproducibility
)

# Train the model
model.fit(
    X_train_prepared, 
    y_train, 
    eval_set=[(X_val_prepared, y_val)], 
    eval_metric='rmse',        # Evaluation metric (root mean squared error)
    callbacks=[lgb.early_stopping(25)]  # Stop early if no improvement
)

In [ ]:
import matplotlib.pyplot as plt
# Plot feature importance
lgb.plot_importance(model, max_num_features=min(X_train_prepared.shape[1], 25), importance_type='gain')
plt.show()

In [ ]:
# Predict on test set
y_test_pred = model.predict(X_test_prepared)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Calculate RMSE
mape = mean_absolute_percentage_error(y_test, y_test_pred)
print("MAPE:", mape)

In [ ]:
# Work Dataset
# RMSE: 1.05268 (FrequencyEncoder)
# RMSE: 1.05283 (OneHotEncoder)

In [ ]:
# Home Dataset Gridsearch
# RMSE: 1.04426 (Freq Encode)   Submit Result =
# RMSE: 1.04422 (OneHot Encode) Submit Result = 1.06506

In [ ]:
# Home Dataset BayesSearch 
# RMSE: 1.04398 (FrequencyEncoder)   Submit Result =
# RMSE: 1.04364 (OneHot Encode): Submit Result = 1.04534

In [ ]:
# Create final predictions
df_solve_cleaned = clean_dataframe(df_solve, impute_num_nulls = False, fill_cat_nulls=True)
df_solve_prepared = transform_dataframe(df_solve_cleaned)
X_solve_prepared, X_solve_prepared_df = pipeline_transform(df_solve_prepared[best_cols], pipeline,
                                                           cat_encoder = cat_encoder_type)


y_solve = model.predict(X_solve_prepared)

df_y_solve = pd.DataFrame(y_solve, columns=[target_variable])
df_final = pd.merge(df_solve_id, df_y_solve, left_index=True, right_index=True, how='inner')
df_final.to_csv('Predictions_LGBM.csv', index=False)

df_final[target_variable].hist(bins=100)